In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import wrangle
import env
import seaborn as sns
import acquire
import prep

In [34]:
#single family residence from zillow DB
df = acquire.wrangle_zillow()

In [35]:
#drops columns with < 80% of values filled in
df = prep.drop_columns(df)

In [36]:
df.lotsizesquarefeet.isnull().sum()

300

(55686, 32)

In [7]:
df.isnull().sum().sort_values(ascending=False)

regionidcity                    1008
lotsizesquarefeet                300
censustractandblock              263
finishedsquarefeet12             186
taxamount                         71
regionidzip                       64
structuretaxvaluedollarcnt        62
yearbuilt                         27
calculatedbathnbr                 20
fullbathcnt                       20
bathroomcnt                        0
bedroomcnt                         0
id                                 0
bathrooms                          0
parcelid                           0
bedrooms                           0
calculatedfinishedsquarefeet       0
tax_value                          0
sqft                               0
id                                 0
latitude                           0
fips                               0
longitude                          0
propertycountylandusecode          0
propertylandusetypeid              0
rawcensustractandblock             0
regionidcounty                     0
r

In [14]:
df[['landtaxvaluedollarcnt','lotsizesquarefeet']].head()

,landtaxvaluedollarcnt,lotsizesquarefeet
0,74104.0,9826.0
1,508238.0,NaN
2,436568.0,6490.0
3,61431.0,6021.0
4,155851.0,4917.0


In [37]:
df['tax_value_per_foot'] = df.landtaxvaluedollarcnt/df.lotsizesquarefeet
df.tax_value_per_foot.mean()

39.52847814507253

In [39]:
df.lotsizesquarefeet.head()

0     9826.000000
1    12857.514983
2     6490.000000
3     6021.000000
4     4917.000000
Name: lotsizesquarefeet, dtype: float64

In [20]:
df.landtaxvaluedollarcnt /df.tax_value_per_foot.mean()

0          1874.699039
1         12857.514983
2         11044.391803
3          1554.094741
4          3942.752348
5          4367.812982
6         17721.957254
7          6888.223700
8          6320.076353
9         11521.769149
10        19440.718086
11        10918.988543
12        25667.013951
13        11702.373117
14        22134.168606
15        15340.383148
16        14784.050068
17         9132.479087
18        10581.940404
19          432.220030
20       101192.866199
21          373.831746
22          644.370874
23         5855.955272
24         1086.001840
25         2285.213199
26         2770.154712
27         6549.404686
28         7805.410541
29         8631.624996
             ...      
55656       522.762347
55657       651.555567
55658       990.450476
55659       851.386180
55660      1373.693159
55661       765.119261
55662      1674.741936
55663       764.967472
55664       146.071902
55665       719.152402
55666       307.853997
55667       521.826313
55668      